# Runner for the Experiments

- runs number of Experiments
- saves files to given path
- one file for each layouting method

In [1]:
%run experiment_tools.ipynb

In [2]:
import time
import pandas as pd
from qiskit import transpile, execute
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import CSPLayout
from qiskit.test.mock.backends import FakeManhattan
from qiskit.providers.aer import QasmSimulator


/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/unittest2/compatibility.py:143: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  class ChainMap(collections.MutableMapping):


## Parameters

In [3]:
exp_name = "exp-graphstate-ring-csplayout"
exp_path = "~/Datastore/qiskit-terra/"
exp_layout_pass = CSPLayout

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
num_repititions = 1
num_shots = 8192
activate_noise = False
num_qubits = 12

In [5]:
# backend
backend = FakeManhattan()

# simulator
simulator_options={"method": "statevector",
                   "precision": "double"}
if activate_noise:
    simulator = QasmSimulator.from_backend(backend)
else: 
    simulator = QasmSimulator()
simulator.set_options(**simulator_options)

## Experiment Loop

In [6]:
df = pd.DataFrame(columns=["method", "time", "num_shots", "noise", "num_qubits", "counts"])

In [8]:
for rep in range(num_repititions):

    # parameters
    coupling_map = CouplingMap(backend.configuration().coupling_map)
    backend_props = backend.properties()
    circuit = graphstate_manhatten_ring(num_qubits)
    
    # time layout selector
    time_start = time.time()
    layout = generate_layout(circuit, exp_layout_pass, coupling_map=coupling_map)
    time_stop = time.time()
    time_delta = time_stop - time_start

    # run transpile
    circuit_t = transpile(circuit, backend, initial_layout=layout, optimization_level=0)

    # execute and get results
    job = execute(circuit_t, simulator, shots=num_shots)
    counts = job.result().get_counts()

    df.loc[len(df)] = [exp_layout_pass(None).name(), time_delta, num_shots, activate_noise, num_qubits, counts]

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
df.to_parquet(exp_path + exp_name)

/Applications/anaconda3/envs/qiskit-terra-dev/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
